<a href="https://colab.research.google.com/github/reanqbow/MAS552/blob/master/FlexiStreet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FlexiStreet**
## *Designing and Visualizing Proactive Street Usage in Post-pandemic MIT-Kendall Square*

Team Member: Clara He, Yubo Zhao, George Guida, Elyjana Roach

GitHub GIST: https://gist.github.com/reanqbow/58b1eda07464d1ab320f53d0382e3f48

@ MIT Media Lab City Science Group


---

# 1. Introduction
## 1.1 Abstract
**ADD NOTES!!!**
## 1.2 Keywords
Responsive Street, Data Visualization, Urban Policy, Sustainability, Lightweight Mobility, Traffic Simulation

## 1.3 Modules
*   Traffic Data: Along with our research on recent commute demand data, [SUMO](https://sumo.dlr.de/docs/index.html) is used to run an agent-based simulation that demonstrates vehicle travel info, pedestrian behaviors, spatial occupancies,and greenhouse gases emissions, in different scenarios.
*   Data Visualization: [Plotly](https://plotly.com/python/) and its integrated [Mapbox](https://plotly.com/python/mapbox-layers/) library are used to visualize the data in both 3D street view and statistical graphs.
*   FlexiStreet Analysis Module: Our own script that analyzes the outcomes based on four metrics.


# 2. System Prep

**ADD NOTES!!!**

## 2.1Installing Dependencies

In [10]:
# Installing SUMO
!pip install eclipse-sumo

In [11]:
# Installing All Python Libraries Needed for Visualization and Data Processing
!pip install plotly==5.4.0
!pip install gwpy
!pip install pydeck

from google.colab import data_table

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

data_table.enable_dataframe_formatter()
px.set_mapbox_access_token("pk.eyJ1IjoibGlhbXoiLCJhIjoiY2oxZnE5MHFhMDByeTJybzJrZjlvYTR4YyJ9.qwj-DfGjKw0dchnd5cNNDQ")
style="mapbox://styles/liamz/ckw3bsrr909vo14mybm880hq7"

## 2.2Set Up Environment Variables

In [12]:
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
%env PYTHONPATH = 
! echo $SUMO_HOME

env: SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
env: PYTHONPATH=
/usr/local/lib/python3.7/dist-packages/sumo


In [13]:
import os, sys
sys.path.append('/usr/local/lib/python3.7/dist-packages/sumo/tools')

In [14]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.7/dist-packages/sumo/tools',
 '/usr/local/lib/python3.7/dist-packages/sumo/tools']

# 3. Street Simulation in SUMO

**ADD NOTES!!!**




In [24]:
#@title Define the Selected Area

map_boundary_xmin = -71.0952 #@param {type:"number"}
map_boundary_ymin = 42.3594 #@param {type:"number"}
map_boundary_xmax = -71.0698 #@param {type:"number"}
map_boundary_ymax = 42.3690 #@param {type:"number"}

In [15]:
#@title Define the Lane Network

net_boundary_xmin = -71.09011 #@param {type:"number"}
net_boundary_ymin = 42.36228 #@param {type:"number"}
net_boundary_xmax = -71.08430 #@param {type:"number"}
net_boundary_ymax = 42.36473 #@param {type:"number"}

In [16]:
type(net_boundary_ymin)

float

In [17]:
# Using SUMO as a Python Library
import sumolib

#!python /usr/local/lib/python3.7/dist-packages/sumo/tools/osmWebWizard.py

In [25]:
#downloading the osm file of kendall
! wget -O kendall.osm "https://api.openstreetmap.org/api/0.6/map?bbox={map_boundary_xmin},{map_boundary_ymin},{map_boundary_xmax},{map_boundary_ymax}"

--2021-12-02 00:08:36--  https://api.openstreetmap.org/api/0.6/map?bbox=-71.0952,42.3594,-71.0698,42.369
Resolving api.openstreetmap.org (api.openstreetmap.org)... 130.117.76.11, 130.117.76.12, 130.117.76.13, ...
Connecting to api.openstreetmap.org (api.openstreetmap.org)|130.117.76.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/xml]
Saving to: ‘kendall.osm’

kendall.osm             [            <=>     ]   6.42M  2.85MB/s    in 2.3s    

2021-12-02 00:08:41 (2.85 MB/s) - ‘kendall.osm’ saved [6735247]



##Building Simulation Network

###Original Network

In [26]:
%%capture
#osm to sumo net
! netconvert --osm-files /content/kendall.osm  --geometry.remove --ramps.guess --junctions.join --tls.guess-signals --tls.discard-simple --tls.join --tls.default-type actuated -o kendall.net.xml --remove-edges.isolated true --type-files /usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvertPedestrians.typ.xml,/usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvertBicycle.typ.xml,/usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvert.typ.xml
#--osm.stop-output.length 25 --ptstop-output kendallPTadditional.xml --ptline-output kendallPTlines.xml
#--junctions.join-dist 10
#osmNetconvert.typ.xml

In [ ]:
#converting kendall.osm to sumo readable file
#!python3 /usr/local/lib/python3.7/dist-packages/sumo/tools/osmBuild.py --osm-file kendall.osm --pedestrians -p "kendall" --netconvert-typemap /usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvert.typ.xml,/usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvertPedestrians.typ.xml,/usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvertBicycle.typ.xml

In [27]:
tree = ET.parse('/content/kendall.net.xml')
root=tree.getroot()
children_list=root.getchildren()

#correction of original network
for edge in root.findall('edge'):
  if edge.get('id')=='90758903#2':
    #print(edge.get('id'))
    for lane in edge:
      if 'allow'in lane.attrib:
        del lane.attrib['allow']
        lane.attrib['disallow'] = 'tram rail_urban rail rail_electric rail_fast ship'
tree.write('kendall.net.xml')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  This is separate from the ipykernel package so we can avoid doing imports until


###Minimum Automobile Lane Network

In [ ]:
#reducing automobile lanes on selected network
#! netconvert -s kendall.net.xml --keep-edges.in-geo-boundary net_boundary_xmin,net_boundary_ymin,net_boundary_xmax,net_boundary_ymax --remove-edges.isolated true -o kendallSelect.net.xml
! netconvert -s kendall.net.xml --keep-edges.in-geo-boundary -71.09011,42.36228,-71.08430,42.36473 --remove-edges.isolated true -o kendallSelect.net.xml
#--edges.join true --join-lanes true

Success.


In [ ]:
subtree = ET.parse('/content/kendallSelect.net.xml')
subtree_root=subtree.getroot()
subtree_children_list=subtree_root.getchildren()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



In [ ]:
'''Run for subnet'''
sub_edges = []
for edge in subtree_root.findall('edge'):
  if len(edge)>3:
    sub_edges.append(edge.get('id'))

In [ ]:
tree = ET.parse('/content/kendall.net.xml')
root=tree.getroot()
children_list=root.getchildren()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



In [ ]:
'''Run for whole net'''
for edge in root.findall('edge'):
  if edge.get('id') in sub_edges and len(edge)>3:
    #print(edge.get('id'))
    for lane in edge:
      if lane.get('index') == '0':
        continue
      elif lane.get('index') == '1':
        lane.set('allow','pedestrian')
      elif lane.get('index') == '2':
        lane.set('allow','bicycle')

tree.write('kendallMinVeh.net.xml')

###No Automobile Lane Network

In [ ]:
'''Run for subnet'''
sub_edges = []
for edge in subtree_root.findall('edge'):
    sub_edges.append(edge.get('id'))

In [ ]:
'''Run for whole net'''
for edge in root.findall('edge'):
  if edge.get('id') in sub_edges:
    for lane in edge:
      if 'disallow'in lane.attrib:
        del lane.attrib['disallow']
        lane.attrib['allow'] = 'pedestrian delivery bicycle'

tree.write('kendallNoVeh.net.xml')

## Demand Modelling

### Senario I: Weekday Peak Hours


####Original Network

#####Vehicles
generate random trips in the network with randomTrips.py

cars-weekday peak hours (morning) original

number of cars: Kendall Square Mobility Data Report:before Covid, peak hour average 8000 cars/hour at Roadway Cordon Locations https://storymaps.arcgis.com/stories/44ecfce9affc4dcca05007cce348eff2#ref-n-rUTU_o

max speed of car is set to 20 mph according to the [speed limit posted by the City of Cambridge](https://www.cambridgema.gov/StreetsAndTransportation/PoliciesOrdinancesAndPlans/VisionZero/SpeedLimitsInCambridge).

fringe factor: According to ["PTDM Survey Results: Kendall Square Area Employee Commute Trip Origins"](https://storymaps.arcgis.com/collections/2bdfc482220c46928d6e30de69dd1c2f?item=1)...Percent of Total Commuters Using SOV From Zip Code located in Kendall square is 11.33%. Therefore, the fringe factor can be estimated to be (1-11.33%)/11.33% = 8

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
 -b 0 -e 1200\
 -p 2.3\
 --binomial 20\
 --fringe-factor 8 -L\
 --vehicle-class passenger\
 --min-distance 1000\
 --trip-attributes="maxSpeed=\"20\""\
 --prefix "car"\
 -r S1Original_Veh.rou.xml\
 -o S1Original_Veh.trips.xml\
 --validate

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakVeh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakVeh.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakVeh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakVeh.trips.xml.tmp --write-trips
Success.


#####Buses
bus-weekday peak hours original

number of buses: es enter into kendall square in 20 minutes? p=1200/x  (subsitute the number x with the answer)

max speed of car is set to 20 mph according to the [speed limit posted by the City of Cambridge](https://www.cambridgema.gov/StreetsAndTransportation/PoliciesOrdinancesAndPlans/VisionZero/SpeedLimitsInCambridge).

fringe factor: no buses depart within kendall square, therefore the fringe factor is set to 100.

bus frequency: https://cdn.mbta.com/sites/default/files/route_pdfs/2021-fall/068-04.21.2.0.pdf

In [ ]:
# !python /usr/local/lib/python3.7/dist-packages/sumo/tools/ptlines2flows.py\
#  -n kendall.net.xml\
#  -b 0 -e 1200\
#  -s kendallPTadditional.xml\
#  -l kendallPTlines.xml\
#  -p 1200\
#  --use-osm-routes\
#  --extend-to-fringe\
#  --vtype-prefix "bus"\
#  -o peakBus.rou.xml\
#  -t peakBus.trips.xml

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
 -b 0 -e 1200\
 -p 600\
 --binomial 5\
 --fringe-factor 100 -L\
 --vehicle-class bus\
 --trip-attributes="maxSpeed=\"20\""\
 --prefix "bus"\
 -r S1Original_Bus.rou.xml\
 -o S1Original_Bus.trips.xml\
 --validate

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakBus.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakBus.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakBus.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakBus.trips.xml.tmp --write-trips
Success.


#####Pedestrians
https://news.mit.edu/2021/counting-pedestrian-traffic-0415

In [ ]:
#pedestrians
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
  -b 0 -e 1200\
  -p 1.3\
  --binomial 20\
  --fringe-factor 0.6\
  --pedestrians --persontrips\
  --vehicle-class pedestrian\
  --trip-attributes="modes=\"public\""\
  --prefix "pedestrian"\
  -r S1Original_ped.rou.xml\
  -o S1Original_ped.trips.xml\
  --validate

# max speed of pedestrian is set to 4mph according to https://en.wikipedia.org/wiki/Walking#:~:text=Specific%20studies%20have%20found%20pedestrian,%2Fh%20(4.0%20mph).

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r ped.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o ped.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r ped.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o ped.trips.xml.tmp --write-trips
Success.


#####Bike
https://storymaps.arcgis.com/stories/99b272fc25b24116949872bef72b0aea#ref-n-WHAprX

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
 -b 0 -e 1200\
 -p 5\
 --binomial 20\
 --fringe-factor 5 -L\
 --vehicle-class bicycle\
 --min-distance 500\
 --trip-attributes="maxSpeed=\"17\""\
 --prefix "bike"\
 -r S1Original_Bike.rou.xml\
 -o S1Original_Bike.trips.xml\
 --validate

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakBike.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakBike.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakBike.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakBike.trips.xml.tmp --write-trips
Success.


####Minimum Vehicle Lane Network

In [ ]:
#Cars
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendallMinVeh.net.xml\
 -b 0 -e 1200\
 -p 2.3\
 --binomial 20\
 --fringe-factor 8 -L\
 --vehicle-class passenger\
 --min-distance 1000\
 --trip-attributes="maxSpeed=\"20\""\
 --prefix "car"\
 -r S1MinVeh_Veh.rou.xml\
 -o S1MinVeh_Veh.trips.xml\
 --validate

 #Buses
 !python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
 -b 0 -e 1200\
 -p 600\
 --binomial 5\
 --fringe-factor 100 -L\
 --vehicle-class bus\
 --trip-attributes="maxSpeed=\"20\""\
 --prefix "bus"\
 -r S1MinVeh_Bus.rou.xml\
 -o S1MinVeh_Bus.trips.xml\
 --validate

#pedestrians
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
  -b 0 -e 1200\
  -p 1.3\
  --binomial 20\
  --fringe-factor 0.6\
  --pedestrians --persontrips\
  --vehicle-class pedestrian\
  --trip-attributes="modes=\"public\""\
  --prefix "pedestrian"\
  -r S1MinVeh_ped.rou.xml\
  -o S1MinVeh_ped.trips.xml\
  --validate

  #Bike
  !python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
 -b 0 -e 1200\
 -p 5\
 --binomial 20\
 --fringe-factor 5 -L\
 --vehicle-class bicycle\
 --min-distance 500\
 --trip-attributes="maxSpeed=\"17\""\
 --prefix "bike"\
 -r S1MinVeh_Bike.rou.xml\
 -o S1MinVeh_Bike.trips.xml\
 --validate


calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendallMinVeh.net.xml -r S1MinVeh_Veh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o S1MinVeh_Veh.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendallMinVeh.net.xml -r S1MinVeh_Veh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o S1MinVeh_Veh.trips.xml.tmp --write-trips
Success.


####No Vehicle Lane Network

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendallNoVeh.net.xml\
 -b 0 -e 1200\
 -p 2.3\
 --binomial 20\
 --fringe-factor 8 -L\
 --vehicle-class passenger\
 --min-distance 1000\
 --trip-attributes="maxSpeed=\"20\""\
 --prefix "car"\
 -r S1NoVeh_Veh.rou.xml\
 -o S1NoVeh_Veh.trips.xml\
 --validate

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendallNoVeh.net.xml -r S1NoVeh_Veh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o S1NoVeh_Veh.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendallNoVeh.net.xml -r S1NoVeh_Veh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o S1NoVeh_Veh.trips.xml.tmp --write-trips
Success.


##Build Configuration

In [ ]:
#S1_Original
!sumo -n kendall.net.xml -r S1Original_Bike.trips.xml, S1Original_Bus.trips.xml, S1Original_ped.trips.xml, S1Original_Veh.trips.xml -C S1Original.sumocfg

In [ ]:
#S1_MinVeh
!sumo -n kendallMinVeh.net.xml -r S1MinVeh_Bike.trips.xml, S1MinVeh_Bus.trips.xml, S1MinVeh_ped.trips.xml, S1MinVeh_Veh.trips.xml -C S1_MinVeh.sumocfg

In [ ]:
#S1_NoVeh
#!sumo -n kendallNoVeh.net.xml -r S1NoVeh_Veh.trips.xml -C S1_NoVeh.sumocfg

##Outputs

In [ ]:
#run simulation and export fcd with geo coordinates

#fcd output
!sumo -c kendallTEST.sumocfg --fcd-output kendallFCD.xml --fcd-output.geo true

Loading configuration ... done.
Step #1511.00 (0ms ?*RT. ?UPS, vehicles TOT 363 ACT 0 BUF 0)                              


In [ ]:
!sumo -c kendallTEST.sumocfg --fcd-output kendallFCD_veh_only.xml --fcd-output.geo true --fcd-output.params vehicle

Loading configuration ... done.
Step #1511.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 363 ACT 0 BUF 0)                 


In [ ]:
#Basic stats output
!sumo -c kendallTEST.sumocfg --statistic-output kendallStats.xml

Loading configuration ... done.
Step #1511.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 363 ACT 0 BUF 0)                 


In [ ]:
#Tripinfo output
!sumo -c kendallTEST.sumocfg --tripinfo-output kendallTripInfoCO2.xml --device.emissions.probability 1
#The complete amount of CO2 emitted by the vehicle during the trip (unit:mg)

Loading configuration ... done.
Step #1511.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 363 ACT 0 BUF 0)                 


In [ ]:
#Emission output
!sumo -c kendallTEST.sumocfg --emission-output kendallEmission.xml

Loading configuration ... done.
Step #1511.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 363 ACT 0 BUF 0)                 


# 4. Analysis Module

Metrics:
1. Mobility
2. Socialability
3. Profitability
4. Sustainability

Scenarios:
**ADD NOTES**

## 4.1 Mobility

## 4.2 Socialability

## 4.3 Profitability

### 4.3.1 An Overview of Outdoor Dining in Cambridge

Data Source: [City of Cambridge Open Data](https://data.cambridgema.gov/Licensing/Outdoor-Dining/qzsr-xrig)

In [ ]:
import json

outdoor_d = pd.read_json("https://data.cambridgema.gov/resource/qzsr-xrig.json")

display(outdoor_d)

fig1 = px.scatter_mapbox(outdoor_d, lat="latitude", lon="longitude", color="typepublicarea", hover_name="doingbusinessas", labels={"typepublicarea":"Outdoor Dining Option"}, color_discrete_map={"Not Applicable":"silver", "Assumed Sidewalk":"peachpuff", 
                                                                                                                                              "Parking Lane":"pink", "Sidewalk": "orange", "Other":"goldenrod", "Road": "salmon", "Parking Lane/Assumed Sidewalk":"magenta", 
                                                                                                                                              "Road + Sidewalk":"palevioletred", "Parking Lane + Sidewalk":"green", "Parking Lot": "teal"}, zoom=15, center={"lat":42.3626, "lon":-71.0984}, width=2000, height=1000)


# fig2 = px.pie(outdoor_d, names="doingbusinessas", values="typepublicarea", color="typepublicarea", hover_name="doingbusinessas", labels={"typepublicarea":"Outdoor Dining Option"}, color_discrete_map={"Not Applicable":"silver", "Assumed Sidewalk":"peachpuff", 
#                                                                                                                                               "Parking Lane":"pink", "Sidewalk": "orange", "Other":"goldenrod", "Road": "salmon", "Parking Lane/Assumed Sidewalk":"magenta", 
#                                                                                                                                               "Road + Sidewalk":"palevioletred", "Parking Lane + Sidewalk":"green", "Parking Lot": "teal"})
fig1.show()

,doingbusinessas,corporatename,fulladdress,latitude,longitude,neighborhood,areatype,typepublicarea,alcohol,website
0,Royal Sonesta Hotel,Sonesta International Hotels Corp.,"40 Land Blvd, Cambridge, MA 02142",42.3669,-71.0749,East Cambridge,Private,Not Applicable,True,{'url': 'https://www.sonesta.com/us/massachuse...
1,Hotel Veritas,"Veritas in the Square, LLC","1 Remington St, Cambridge, MA 02138",42.3711,-71.1138,Mid-Cambridge,Private,Not Applicable,True,{'url': 'https://www.thehotelveritas.com/'}
2,Hotel Marlowe and Bambara,"Cambridge Lodging, LLC","25 Land Blvd, Cambridge, MA 02141",42.3678,-71.0749,East Cambridge,Private,Not Applicable,True,{'url': 'https://www.bambara-cambridge.com/'}
3,Sheraton Commander,"Commander Properties, Inc.","16 Garden St, Cambridge, MA 02138",42.3773,-71.1232,West Cambridge,Private,Not Applicable,True,{'url': 'https://www.nubarcambridge.com/'}
4,AC Hotels by Marriott-Cambridge,"Colwen Management, Inc.","10 Acorn Park Dr, Cambridge, MA 02140",42.3987,-71.1473,North Cambridge,Private,Not Applicable,True,{'url': 'https://www.marriott.com/hotels/trave...
...,...,...,...,...,...,...,...,...,...,...
249,Muku,TTT&K Corp.,"411 Massachusetts Ave, Cambridge, MA 02139",42.3636,-71.1002,The Port,Private,Not Applicable,False,{'url': 'https://www.mukuramen.com/'}
250,Middlesex,Rush Hour Group LLC,"315 Massachusetts Ave, Cambridge, MA 02139",42.3625,-71.0985,The Port,Public,Assumed Sidewalk,True,{'url': 'http://www.middlesexlounge.us/'}
251,Cloud and Spirits,Love Art Udon Comm LLC,"795 Main St, Cambridge, MA 02139",42.3633,-71.0971,The Port,Public,Parking Lane,True,{'url': 'https://www.cloudandspirits.com/'}
252,Orale Paco's Mexican Grill,Orale Mexican Grill Inc,"847 Cambridge St, Cambridge, MA 02141",42.3725,-71.0909,Wellington-Harrington,Public,Parking Lane,False,{'url': 'http://www.oralemexicangrill.com/'}


In [ ]:
from bs4 import BeautifulSoup 

# For website connections
import requests 

# To prevent overwhelming the server between connections
from time import sleep 

# Display the progress bar
from tqdm import tqdm

# For data wrangling
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## 4.4 Sustainability

# 5. Visualization

In [ ]:
#Parse XML output

data_table.enable_dataframe_formatter()

tree1 = ET.parse('/content/kendallTripInfoCO2.xml')
root1 = tree1.getroot()


em_df_cols = ["ID", "TRAVEL_TIME", "CO", "CO2", "HC", "PMx", "NOx",  ]
em_rows = []

for child in root1:
  if child.tag == 'tripinfo':
    travel_time = child.get('duration')
    id = child.get('id')
    co = float(child[0].get('CO_abs'))
    co2 = float(child[0].get('CO2_abs'))
    hc = child[0].get('HC_abs')
    pmx = child[0].get('PMx_abs')
    nox = child[0].get('NOx_abs')
    #avg_co2 = float(child[0].get('CO2_abs')) / 4
    em_rows.append({"ID": id, "TRAVEL_TIME": travel_time, "CO": co, "CO2": co2, "HC": hc, "PMx": pmx, "NOx": nox})

em_out_df = pd.DataFrame(em_rows, columns = em_df_cols)

display(em_out_df)


,ID,TRAVEL_TIME,CO,CO2,HC,PMx,NOx
0,car6,178.00,11462.588718,5.542037e+05,65.312639,10.664683,231.299019
1,car5,183.00,8120.830029,3.962204e+05,46.248233,7.029290,161.604658
2,car2,209.00,11660.811056,4.954878e+05,64.645558,9.070805,204.268346
3,car3,209.00,7665.454385,4.270251e+05,44.793879,6.925817,170.222517
4,car15,188.00,10334.186639,5.113571e+05,59.090155,9.615259,212.466572
...,...,...,...,...,...,...,...
358,car500,343.00,31665.816483,8.943989e+05,165.227295,18.904144,386.883878
359,car492,363.00,30346.897601,1.055129e+06,163.021557,21.442493,447.721525
360,car471,416.00,24201.152776,9.668258e+05,132.752233,17.908200,401.076704
361,car493,365.00,26641.811753,9.426823e+05,143.487955,18.969589,399.400782


In [ ]:
# Parse Veh-only output

tree2 = ET.parse('/content/kendallFCD.xml')
root2 = tree2.getroot()

for child in root2:
  for person in child.findall('person'):
    child.remove(person)

tree2.write('/content/veh_only_output.xml')

In [ ]:
def readParser(xmlTree):
  result = {}
  root = xmlTree.getroot()

  for child in root:
    time = child.get('time')
    for veh in child:
      vehid = veh.get('id')
      veh_xpos = veh.get('x')
      veh_ypos = veh.get('y')

      if vehid in result:
        result[vehid].append([time, veh_xpos, veh_ypos])
      else:
        result[vehid] = [[time, veh_xpos, veh_ypos]]

  return result

def start_end_only(original):
  
  for key in original:
    length = len(original[key])
    newlst = original[key][0]+original[key][length//3]+original[key][2*length//3]+original[key][-1]
    original[key] = newlst
    
  return original

In [ ]:
res = readParser(tree2)

res2 = start_end_only(res)

trip_df_cols = ["ID", "START_TIME","END_TIME", "START_X", "START_Y", "X1", "Y1", "X2", "Y2", "END_X", "END_Y"]
trip_rows = []

for key in res2:
  start_time = float(res2[key][0])
  start_x = float(res2[key][1])
  start_y = float(res2[key][2])
  x1 = float(res2[key][4])
  y1 = float(res2[key][5])
  x2 = float(res2[key][7])
  y2 = float(res2[key][8])

  end_time = float(res2[key][9])
  end_x = float(res2[key][10])
  end_y = float(res2[key][11])
  
  
  trip_rows.append({"ID":key, "START_TIME": start_time, "END_TIME": end_time, "START_X": start_x, "START_Y": start_y, "X1":x1, "Y1":y1, "X2":x2, "Y2":y2, "END_X": end_x, "END_Y": end_y})

trip_out_df = pd.DataFrame(trip_rows, columns = trip_df_cols)

final_out_df = pd.merge(em_out_df, trip_out_df, on='ID')
display(final_out_df)

NameError: ignored

In [ ]:
overwrite_df_cols = ["ID", "CO2", "X", "Y"]
overwrite_df_rows = []

for i in range(final_out_df.shape[0]):
  id = final_out_df.iloc[i, 0]
  co2 = float(final_out_df.iloc[i, 3])/4

  x1 = final_out_df.iloc[i, 9]
  y1 = final_out_df.iloc[i, 10]
  overwrite_df_rows.append({"ID": id, "CO2": co2, "X": x1, "Y": y1})
  x2 = final_out_df.iloc[i, 11]
  y2 = final_out_df.iloc[i, 12]
  overwrite_df_rows.append({"ID": id, "CO2": co2, "X": x2, "Y": y2})
  x3 = final_out_df.iloc[i, 13]
  y3 = final_out_df.iloc[i, 14]
  overwrite_df_rows.append({"ID": id, "CO2": co2, "X": x3, "Y": y3})
  x4 = final_out_df.iloc[i, 15]
  y4 = final_out_df.iloc[i, 16]
  overwrite_df_rows.append({"ID": id, "CO2": co2, "X": x4, "Y": y4})

overwrite_df = pd.DataFrame(overwrite_df_rows, columns=overwrite_df_cols)

display(overwrite_df)

,ID,CO2,X,Y
0,car6,138550.924523,-71.080022,42.364042
1,car6,138550.924523,-71.082293,42.362128
2,car6,138550.924523,-71.087955,42.362577
3,car6,138550.924523,-71.093811,42.361882
4,car5,99055.098380,-71.077979,42.365048
...,...,...,...,...
1447,car493,235670.585530,-71.094121,42.359590
1448,car494,270181.232476,-71.073852,42.369481
1449,car494,270181.232476,-71.078635,42.365238
1450,car494,270181.232476,-71.083623,42.363642


In [ ]:

px.set_mapbox_access_token("pk.eyJ1IjoibGlhbXoiLCJhIjoiY2oxZnE5MHFhMDByeTJybzJrZjlvYTR4YyJ9.qwj-DfGjKw0dchnd5cNNDQ")

fig = ff.create_hexbin_mapbox(
    data_frame=overwrite_df, lat="Y", lon="X",
    nx_hexagon=40, opacity=0.4, labels={"color": "Cumulated CO2 Emission"},
    color="CO2",agg_func=np.sum, color_continuous_scale=px.colors.sequential.dense, height=800, min_count=0, show_original_data=True,
    original_data_marker=dict(size=4, opacity=0.6, color="deeppink"))
fig.show()

In [ ]:


px.set_mapbox_access_token("pk.eyJ1IjoibGlhbXoiLCJhIjoiY2oxZnE5MHFhMDByeTJybzJrZjlvYTR4YyJ9.qwj-DfGjKw0dchnd5cNNDQ")
df = px.data.carshare()

fig = create_hexbin_mapbox(
    data_frame=df, lat="centroid_lat", lon="centroid_lon",
    nx_hexagon=10, opacity=0.9, labels={"color": "Point Count"},
)
fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
fig.show()

TypeError: ignored

In [ ]:


# def line_array(df, cols):
#     return np.pad(df.loc[:,cols].values, [(0, 0), (0, 1)], constant_values=None).reshape(1,(len(df)*3))[0]
    
df = final_out_df

# SX = df['START_X']
# SY = df['START_Y']
# EX = df['END_X']
# EY = df['END_Y']

# fig = px.line_mapbox(lat=[SY, EY], 
#                      lon=[SX, EX],
# )

# fig.update_layout(mapbox_style="stamen-terrain",
#                   mapbox_zoom=4,
#                   mapbox_center_lon=df.loc[:,["START_X","END_X"]].mean().mean(),
#                   mapbox_center_lat=df.loc[:,["START_Y","END_Y"]].mean().mean(),
#                   margin={"r":0,"t":0,"l":0,"b":0}
#                  )

fig = go.Figure()

flight_paths = []
for i in range(len(df)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df['START_X'][i], df['END_X'][i]],
            lat = [df['START_Y'][i], df['END_Y'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
        )
    )

fig.update_layout(
    showlegend = False,
    geo = dict(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()


In [ ]:
px.scatter(data_frame=df, x='TRAVEL_TIME', y='CO2', opacity=0.3)

In [ ]:
!pip install keplergl

     |████████████████████████████████| 9.7 MB 8.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 42.4 MB/s 
     |████████████████████████████████| 6.3 MB 45.4 MB/s 
     |████████████████████████████████| 15.4 MB 40.2 MB/s 
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922343 sha256=c1f0564a203d93e2635365e430a889e73bbb8ea1d575838a8ff4295a8ec4f923
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


In [ ]:
from IPython.display import Javascript
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
from google.colab import output
output.enable_custom_widget_manager()

from keplergl import KeplerGl


map_1 = KeplerGl(height=800, data={"data1": df})
map_1



<IPython.core.display.Javascript object>

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data1':          ID TRAVEL_TIME             CO  ... END_TIME       END_X      END_Y
0      car…